<a href="https://colab.research.google.com/github/chr1shr/am205_g_activities/blob/master/eno_methods/lin_adv_eno_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Linear advection using ENO

In this exercise, we will implement the linear advection example with step function initial condition using the ENO method.

In [1]:
%matplotlib inline
import numpy as np
from math import *
import matplotlib.pyplot as plt

With your group, complete the following function ```eno``` which takes as its input the five values of density $U_{j-2}$, $U_{j-1}$, $U_j$, $U_{j+1}$, $U_{j+2}$ from which the three candidate interpolants $U^{(0)}_{j+1/2}$, $U^{(1)}_{j+1/2}$, $U^{(2)}_{j+1/2}$ may be constructed. Calculate $\beta^{(k)}$ for each candidate and select the $U^{(k)}_{j+1/2}$ with the smallest $\beta^{(k)}$ to return.

In [2]:
# set up the different stencils for easy access.
stencil_coeffs=np.array([[3./8,-10./8,15./8,0,0],
                         [0,-1./8,6./8,3./8,0],
                         [0,0,3./8,6./8,-1./8]])

beta_coeffs=np.array([[4./3,-19./3,25./3,11./3,-31./3,10./3],
                      [4./3,-13./3,13./3,5./3,-13./3,4./3],
                      [10./3,-31./3,25./3,11./3,-19./3,4./3]])
    
# short routine for calculating the smoothness indicator bk.
def smoothness(k,ul,uc,ur):
    c=beta_coeffs[k]
    us=np.array([ul**2,ul*uc,uc**2,ul*ur,uc*ur,ur**2])
    return np.dot(c,us)

def eno(um2,um1,u,up1,up2):
    us = np.array([um2,um1,u,up1,up2])
    '''us - the set of 5 values at [j-2,j-1,j,j+1,j+2] of the density
       from which to construct the interpolated value at j+1/2.
       Complete the function eno, which should compute the smoothness
       indicator beta of each of the three possible stencils from these
       five points, choose the scheme with the lowest beta, and output
       the interpolated value u_j+1/2.
    '''
    # compute the smoothness indicator of each stencil.
    beta=np.zeros(3)
    for k in range(3):
        beta[k]=smoothness(k,us[k],us[k+1],us[k+2])
    opt=np.argmin(beta) # choose scheme with lowest beta
    c=stencil_coeffs[opt]
    return np.dot(c,us) # multiply density by the stencil coefficients

Below, we set up the linear, hyperbolic PDE problem:

$$u_t + cu_x = 0 $$

which describes the motion of a wave $u(x,t)$ with constant speed $c>0$. The initial condition of the wave is given by the sigmoid function

$$ u(x,0) = \frac{1}{1+e^{-kx}} $$
for some sharpness $k$.

In [3]:
# Grid size
m=201
# Pad by 6 ghost nodes: three on left, three on right. 
# For c>0 only one set is used, but we can maintain a
# more general implementation here. Also, we will take
# advantage of Python's negative indexing and put all 6
# ghost nodes at the end of the interval, so the indexing
# across the array will go
# 0,   1,   ...,   j,   ...,   m-1,   m,   m+1,   m+2,   -3,   -2,   -1.
#|_________domain [-1,1]___________||___right ghost___||___left ghost __|
mp6=m+6

# PDE-related constants.
c=1.
dx=2.0/(m-1)
dt=0.001
nu=c*dt/dx
T=1. # final simulation time
snaps=100 # number of snapshots to output (excluding t=0)
iters=int(T/dt)//snaps # iterations to perform between snapshots

u=np.empty(mp6) # memory for the current step
u1=np.empty(mp6) # memory for the next step
us=np.empty((m,snaps+1)) # memory for all snapshots

# Initial condition
k=100
for j in range(-3,m+3,1): # include ghost nodes
    x=-1+dx*j
    u[j]= 1./(1+np.exp(-k*x))

u1=np.copy(u)
us[:,0]=u[:m]

Below, we loop over the number of snapshots, taking ```iters``` number of steps of size $dt$ between each snapshot. At each iteration, we loop over each cell and compute the finite difference update based on the ENO method.

In [4]:
# Integrate the PDE
for i in range(1,snaps+1):
    for k in range(iters):
        for j in range(m):
              ur=eno(u[j-2],u[j-1],u[j],u[j+1],u[j+2]) # u_j+1/2
              ul=eno(u[j-3],u[j-2],u[j-1],u[j],u[j+1]) # u_j-1/2
              u1[j]=u[j]-nu*(ur-ul)
        u=np.copy(u1)
    us[:,i]=u[:m]

The following code cell defines a set of helper functions that allow us to visualize the propagation of the wave as an animation.

In [5]:
import matplotlib.animation as animation

# plot the wave
def plot_wave(x, y, ax):
      line, = ax.plot(x,y,color='C0')
      return line
    
def update_line(line,x,y):
      line.set_data(x,y)

def animate_wave(x,z):
    # each column of z represents a frame.
    # initialize plot 
    fig, ax = plt.subplots(1,1,figsize=(12,4))
    ax.plot(x,z[:,0],linestyle='dashed',color='C0') # initial state.
    line = plot_wave(x,z[:,0],ax)
    ax.set_xlim(-1,1); ax.set_ylim(-0.1,1.1)
    ax.set_xlabel('x'); ax.set_ylabel('u')

    def animate(i):
        '''Plot updates for animation.'''
        fr = i
        update_line(line,x,z[:,i])
        return line,

    ani = animation.FuncAnimation(fig, animate, frames=z.shape[1], interval=100, blit=True)
    plt.close(fig)
    return ani

Animate the wave:

In [6]:
from IPython.display import HTML

x=np.linspace(-1,1,m)
ani = animate_wave(x,us)
HTML(ani.to_html5_video())